## 1.10 从序列中移除重复项且保持元素间顺序不变
### 1.10.1 问题
去除序列中出现的重复的元素，但仍然保持剩下的元素顺序不变。
### 1.10.2 解决方案
如果序列中的值是可哈希(hashable)的，那么可以通过使用集合和生成器轻松解决。

如果一个对象是可哈希的，那么再它的生存期内必须是**不可变的**，它需要有一个`__hash__()`方法。
整数、浮点数、字符串、元组都是不可变的。

In [1]:
def dedupe(items):
    seen = set()
    for item in items:
        if item not in seen:
            yield item
            seen.add(item)

In [2]:
a = [1, 5, 2, 1, 9, 1, 5, 10]
list(dedupe(a))

[1, 5, 2, 9, 10]

只有当序列中的元素是可哈希的时候才能这么做。如果想在不可哈希的对象(比如列表)序列中去除重复项，需要对上述代码稍作修改：

In [10]:
def dedupe(items, key=None):
    seen = set()
    for item in items:
        val = item if key is None else key(item)
        if val not in seen:
            yield item
            seen.add(val)
    print(seen)

In [11]:
# 参数key的作用是指定一个函数用来将序列中的元素转换为可哈希的类型，这么做是为了检测重复项。
a = [{'x':1, 'y':2}, {'x':1, 'y':3}, {'x':1, 'y':2}, {'x':2, 'y':4}]

In [12]:
list(dedupe(a, key=lambda k: (k['x'], k['y']))) # 去除x, y都相同的元素

{(2, 4), (1, 2), (1, 3)}


[{'x': 1, 'y': 2}, {'x': 1, 'y': 3}, {'x': 2, 'y': 4}]

In [13]:
list(dedupe(a, key=lambda k: k['x']))   # 去除x值相同的元素

{1, 2}


[{'x': 1, 'y': 2}, {'x': 2, 'y': 4}]

### 1.10.3 讨论
如果想要做的只是去除重复项，最简单的方法就是构建一个集合。

但是这种方法不能保证元素间的顺序不变，因此得到的结果会被打乱。

In [14]:
a = [1, 5, 2, 1, 9, 1, 5, 10]
set(a)

{1, 2, 5, 9, 10}

## 1.11 对切片命名
### 1.11.1 问题与解决方案
假设有一些代码用来从字符串的固定位置中取出具体的数据。可以对切片命名，避免使用许多神秘难懂的硬编码索引。

In [16]:
record = '....................100.......513.25..........'
cost = int(record[20:23]) * float(record[30:36])
cost

51325.0

In [17]:
# 对切片命名
SHARES = slice(20, 23)
PRICE = slice(30, 36)
cost = int(record[SHARES]) * float(record[PRICE])
cost

51325.0

### 1.11.2 讨论
作为一条基本准则，代码中如果有很多硬编码的索引值，将导致可读性和可维护性都不佳。

一般来说，内置的`slice()`函数会创建一个切片对象，可以用在任何允许进行切片操作的地方。

In [18]:
items = [0, 1, 2, 3, 4, 5, 6]
a = slice(2, 4)
print(items[2:4])
print(items[a])

[2, 3]
[2, 3]


In [19]:
items[a] = [10, 11] # 通过索引改变元素的值
print(items)
del items[a]        # 通过索引删除元素
print(items)

[0, 1, 10, 11, 4, 5, 6]
[0, 1, 4, 5, 6]


In [20]:
# 如果有一个slice对象的实例s，可以通过`s.start, s.stop, s.step`属性来得到关于该对象的信息。
a = slice(10, 50, 2)
print(a.start)
print(a.stop)
print(a.step)

10
50
2


此外，可以通过使用`indices(size)`方法将切片映射到特定大小的序列上。返回一个(start, stop, step)元组，所有的值都已经恰当地限制在边界以内(当做索引操作时可避免出现`IndexError`异常)。 

In [32]:
a = slice(5, 12, 2)
s = 'HelloWorld'
a.indices(len(s))
a

slice(5, 12, 2)

In [33]:
for i in range(*a.indices(len(s))):
    print(s[i])

W
r
d


In [34]:
assert range(*a.indices(len(s))) == range(len(s))[a]

## 1.12 找出序列中出现次数最多的元素
### 1.12.2 解决方案
`collections`模块中的`Counter`类正是为此类问题所设计的。它有一个非常方便的`most_common()`方法可以直接得到答案。

如下有一个列表，列表中是一系列的单词，找出出现最频繁的单词。

In [35]:
words = [
    'look', 'into', 'my', 'eyes', 'look', 'into', 'my', 'eyes',
    'the', 'eyes', 'the', 'eyes', 'the', 'eyes', 'not', 'around', 'the',
    'eyes', "don't", 'look', 'around', 'the', 'eyes', 'look', 'into',
    'my', 'eyes', "you're", 'under'
]
from collections import Counter
word_counts = Counter(words)
top_three = word_counts.most_common(3)
print(top_three)

[('eyes', 8), ('the', 5), ('look', 4)]


### 1.12.3 讨论
可以给`Counter`对象提供任何可哈希的对象序列作为输入。在底层实现中，`Counter`是一个字典，在元素和它们出现的次数间做了映射。

In [36]:
word_counts['not']

1

In [37]:
word_counts['eyes']

8

In [38]:
# 如果想手动增加技术，只需要简单地自增即可：
morewords = ['why', 'are', 'not', 'looking', 'in', 'my', 'eyes']
for word in morewords:
    word_counts[word] += 1

word_counts['eyes']

9

In [39]:
# 另一种方法是使用update()
word_counts.update(morewords)
word_counts['eyes']

10

In [42]:
# `Counter`对象有一个特性，就是它们可以轻松地同各种数学运算操作结合起来使用。
a = Counter(words)
b = Counter(morewords)

In [43]:
a

Counter({'look': 4,
         'into': 3,
         'my': 3,
         'eyes': 8,
         'the': 5,
         'not': 1,
         'around': 2,
         "don't": 1,
         "you're": 1,
         'under': 1})

In [44]:
b

Counter({'why': 1,
         'are': 1,
         'not': 1,
         'looking': 1,
         'in': 1,
         'my': 1,
         'eyes': 1})

In [45]:
# Combine counts
c = a + b
c

Counter({'look': 4,
         'into': 3,
         'my': 4,
         'eyes': 9,
         'the': 5,
         'not': 2,
         'around': 2,
         "don't": 1,
         "you're": 1,
         'under': 1,
         'why': 1,
         'are': 1,
         'looking': 1,
         'in': 1})

In [46]:
# Subtract counts
d = a - b
d

Counter({'look': 4,
         'into': 3,
         'my': 2,
         'eyes': 7,
         'the': 5,
         'around': 2,
         "don't": 1,
         "you're": 1,
         'under': 1})

## 1.13 通过公共键对字典列表排序
### 1.13.1 问题
有一个字典列表，想根据一个或多个字典中地值来对列表排序。
### 1.13.2 解决方案
利用`operator`模块中地`itemgetter`函数对这类结构进行排序非常简单。

假设通过查询数据库表项获取网站上地成员列表，得到如下地数据结构：

In [47]:
rows = [
    {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
    {'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
    {'fname': 'John', 'lname': 'Cleese', 'uid': 1001},
    {'fname': 'Big', 'lname': 'Jones', 'uid': 1004},
]

In [48]:
from operator import itemgetter
rows_by_fname = sorted(rows, key=itemgetter('fname'))
rows_by_uid = sorted(rows, key=itemgetter('uid'))
print(rows_by_fname)
print(rows_by_uid)

[{'fname': 'Big', 'lname': 'Jones', 'uid': 1004}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}, {'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}]
[{'fname': 'John', 'lname': 'Cleese', 'uid': 1001}, {'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}, {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}]


In [49]:
# itemgetter()函数可以接受多个键
rows_by_lfname = sorted(rows, key=itemgetter('lname', 'fname'))
print(rows_by_lfname)

[{'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}, {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}]


In [51]:
# 使用匿名函数也可以实现类似功能
rows_uid = sorted(rows, key=lambda k:k['uid'])
rows_uid

[{'fname': 'John', 'lname': 'Cleese', 'uid': 1001},
 {'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
 {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
 {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}]

### 1.13.3 讨论
上述代码中，rows被传递给内建地`sorted()`函数，该函数接受一个关键字参数`key`。这个参数应该代表一个可调用对象(callable)，该对象从rows中接受一个单独的元素作为输入并返回一个用来做排序依据的值。

`itemgetter()`函数创建的就是这样一个可调用对象。函数`operator.itemgetter()`接受的参数可作为查询的标记，用来从rows的记录中提取出所需要的值。它可以是字典的键名称、用数字表示的列表元素或是任何可以传给对象的`__getitem__()`方法的值。如果传多个标记给`itemgetter()`，那么它产生的可调用对象将返回一个包含所有元素在内的元组，然后`sorted()`将根据对元组的排序结果来排列输出结果。

## 1.14 对不原生支持的比较操作的对象排序
